<a href="https://colab.research.google.com/github/themannnphil/intrusion-detection-system/blob/main/Model_Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import gradio as gr
import joblib
import pandas as pd

In [3]:
!git clone https://github.com/themannnphil/intrusion-detection-system.git
%cd intrusion-detection-system

/content/intrusion-detection-system


In [4]:
%cd models

/content/intrusion-detection-system/models


In [5]:
import joblib
model = joblib.load("xgb_nslkdd_pipeline.joblib")

In [8]:
# Define prediction function
def predict_intrusion(duration, protocol_type, service, flag, src_bytes, dst_bytes,
                      count, srv_count, serror_rate, same_srv_rate):

    # Create dataframe with same feature names as training
    input_data = pd.DataFrame([{
        "duration": duration,
        "protocol_type": protocol_type,
        "service": service,
        "flag": flag,
        "src_bytes": src_bytes,
        "dst_bytes": dst_bytes,
        "count": count,
        "srv_count": srv_count,
        "serror_rate": serror_rate,
        "same_srv_rate": same_srv_rate
    }])

    # Predict (0 = normal, 1 = attack)
    pred = model.predict(input_data)[0]
    label = "🚨 Attack Detected" if pred == 1 else "✅ Normal Traffic"

    # Probability score
    try:
        prob = model.predict_proba(input_data)[0][1]
        label += f" (Attack Probability: {prob:.2f})"
    except:
        pass

    return label

# Gradio UI
demo = gr.Interface(
    fn=predict_intrusion,
    inputs=[
        gr.Number(label="Duration"),
        gr.Dropdown(["tcp", "udp", "icmp"], label="Protocol Type"),
        gr.Dropdown(["http", "ftp", "smtp", "private", "other"], label="Service"),
        gr.Dropdown(["SF", "S0", "REJ", "RSTO", "RSTR", "SH", "OTH"], label="Flag"),
        gr.Number(label="Source Bytes"),
        gr.Number(label="Destination Bytes"),
        gr.Number(label="Count"),
        gr.Number(label="Service Count"),
        gr.Slider(0, 1, step=0.01, label="SYN Error Rate (serror_rate)"),
        gr.Slider(0, 1, step=0.01, label="Same Service Rate (same_srv_rate)")
    ],
    outputs=gr.Textbox(label="Prediction"),
    title="🔐 Intrusion Detection System (IDS) — Demo",
    description="Enter network flow features to check if traffic is normal or an intrusion attempt (XGBoost model)."
)

# Launch
demo.launch()

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://f3741a981c60db138e.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
